In [3]:
import pandas as pd
import re
import numpy as np
from collections import Counter

# Load the Toyota CSV file
toyota_data = pd.read_csv('/home/bipin/Documents/kotak/car_model/manufacturer_data/TOYOTA.csv')
print(f"Loaded {len(toyota_data)} records from TOYOTA.csv")
toyota_data.head()

# Function to clean and normalize model names
def clean_model(text):
    if pd.isna(text) or text is None:
        return ""
    # Convert to string
    text = str(text).upper()
    # Remove common prefixes and company names
    text = re.sub(r'TOYOTA\s+KIRLOSKAR\s+MOTOR\s+PVT\s+LTD,?\s*', '', text)
    text = re.sub(r'TOYOTA\s+KIRLOSKAR\s+MOTORS?\s+LTD,?\s*', '', text)
    text = re.sub(r'TOYOTA\s+KIRLOSKAR\s+MOTOR,?\s*', '', text)
    text = re.sub(r'TOYOT?A\s+', '', text)
    text = re.sub(r'TOYATO\s+', '', text)
    text = re.sub(r'TOTOTA\s+', '', text)
    # Remove special characters and extra spaces
    text = re.sub(r'[^\w\s-]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Create a list of cleaned model names
toyota_data['clean_model'] = toyota_data['rc_maker_model'].apply(clean_model)

# Filter out too generic or empty model names
generic_terms = ['LTD', 'BS', 'IV', 'III', 'BS-IV', 'BS-III', 'BSIV', 'BSIII', 'LIMITED', 'STD', 'CARS']
filtered_models = toyota_data['clean_model'].apply(
    lambda x: x if x and all(term not in x.split() for term in generic_terms) else np.nan
)

# Count model name frequencies
model_counts = Counter([m for models in filtered_models.dropna() for m in models.split()])
model_counts = {k: v for k, v in model_counts.items() if v >= 10}  # Keep only frequent terms

# Print top model keywords by frequency
print("Top model keywords by frequency:")
sorted_counts = sorted(model_counts.items(), key=lambda x: x[1], reverse=True)
for model, count in sorted_counts[:30]:
    print(f"{model}: {count}")

# Define primary Toyota model keywords
toyota_model_keywords = [
    # SUVs and Crossovers
    "FORTUNER", "RAV4", "URBAN", "CRUISER", "HYRYDER",
    
    # MPVs
    "INNOVA", "CRYSTA", "QUALIS",
    
    # Sedans
    "COROLLA", "ALTIS", "CAMRY", "YARIS",
    
    # Hatchbacks
    "ETIOS", "LIVA", "GLANZA",
    
    # Common model combinations
    "INNOVA CRYSTA", "URBAN CRUISER", "COROLLA ALTIS", "ETIOS LIVA", "ETIOS CROSS",
    
    # Trim levels
    "VX", "GX", "ZX", "VXD", "GD", 
    
    # Specific model numbers for Innova
    "2.4G", "2.4V", "2.8G", "2.8Z", "2.5G", "2.5V", "2.5Z", "2.5E", "2.0G", "2.0V",
    
    # Specific model numbers for Fortuner
    "3.0L", "2.8L", "2WD", "4WD", "4X4", "LEGENDER", "SIGMA",
    
    # Specific model numbers for Etios
    "VD", "GD", "VX", "GX", "G", "V", "J",
    
    # Legacy models
    "QUALIS"
]

# Create alias map for common misspellings and variations
toyota_alias_map = {
    # Innova variations
    "INOVA": "INNOVA",
    "INNO": "INNOVA",
    "INVOVA": "INNOVA",
    "INNOVA CRYSTA": "INNOVA",
    "INNOVACRISTA": "INNOVA",
    "INNOVACRYSTA": "INNOVA",
    "INNVOVA": "INNOVA",
    "INNOVVA": "INNOVA",
    "INNOVA CRISTA": "INNOVA",
    "INNOVACRYSTAL": "INNOVA",
    "INNOVA25": "INNOVA",
    "INNOVACRYSTALAD": "INNOVA",
    
    # Etios variations
    "ETOS": "ETIOS",
    "ETOIS": "ETIOS",
    "ETIO": "ETIOS",
    "ETOES": "ETIOS",
    "ETIOUS": "ETIOS",
    "ETOES": "ETIOS",
    "ETIOSTSPORT": "ETIOS",
    "ETIOS LIVA": "ETIOS",
    "ETIOS CROSS": "ETIOS",
    "ETIOS LIVAGD": "ETIOS",
    "ETIOSGD": "ETIOS",
    "ETIOSVD": "ETIOS",
    "ETIOSGDM": "ETIOS",
    "ETIOSVDM": "ETIOS",
    "ETIOSGDF": "ETIOS",
    "ETIOSVDF": "ETIOS",
    
    # Fortuner variations
    "FORTUNERFOUR": "FORTUNER",
    "FORTUNA": "FORTUNER",
    "FORTUNER30L": "FORTUNER",
    "FORTUNER28L": "FORTUNER",
    "FORTUNER4WD": "FORTUNER",
    "FORTUNER2WD": "FORTUNER",
    "FORTUNERSIGMA": "FORTUNER",
    "FORTUNERSIGMA4": "FORTUNER",
    "FORTUNERLEGENDER": "FORTUNER",
    
    # Corolla variations
    "COROLA": "COROLLA",
    "CAROLLA": "COROLLA",
    "CORROLA": "COROLLA",
    "COROLL": "COROLLA",
    "COROLLA ALTIS": "COROLLA",
    "COROLLAALTIS": "COROLLA",
    "COROLAALTIS": "COROLLA",
    "COROLLAATTIS": "COROLLA",
    "COROLLAALTISV": "COROLLA",
    "COROLLAALTISJ": "COROLLA",
    "COROLLAALTISVI": "COROLLA",
    
    # Glanza variations
    "GLANSA": "GLANZA",
    "GLANZA V": "GLANZA",
    "GLANZA G": "GLANZA",
    "GLANZAV": "GLANZA",
    "GLANZAG": "GLANZA",
    
    # Yaris variations
    "YARRIS": "YARIS",
    "YARES": "YARIS",
    "YARIS15": "YARIS",
    "YARISJ": "YARIS",
    "YARISV": "YARIS",
    "YARISG": "YARIS",
    
    # Qualis variations
    "QUAILS": "QUALIS",
    "QUALLIS": "QUALIS",
    "QUALLS": "QUALIS",
    "QUALIS24D": "QUALIS",
    "QUALLIS24D": "QUALIS",
    
    # Urban Cruiser variations
    "URBANCRUISER": "URBAN CRUISER",
    "URBAN CRUIZER": "URBAN CRUISER",
    "URBANCRUIZER": "URBAN CRUISER",
    "URBAN CRUSIER": "URBAN CRUISER",
    
    # Trim level mappings
    "GD M": "GD",
    "GD F": "GD",
    "VD M": "VD",
    "VD F": "VD",
    "VX M": "VX",
    "VX F": "VX",
    "GX M": "GX",
    "GX F": "GX",
    
    # Handle specific combinations
    "INNOVA 25G": "INNOVA",
    "INNOVA 25V": "INNOVA",
    "INNOVA 25Z": "INNOVA",
    "INNOVA 25E": "INNOVA",
    "INNOVA 24G": "INNOVA",
    "INNOVA 24V": "INNOVA",
    "INNOVA 28G": "INNOVA",
    "INNOVA 28Z": "INNOVA",
    "INNOVA 20G": "INNOVA",
    "INNOVA 20V": "INNOVA",
    
    # Models with Non-Toyota names that appear in the data
    "HERO HONDA": "NOT FOUND",
    "TVS XL": "NOT FOUND",
    "A STAR": "NOT FOUND"
}

# Function to normalize text
def normalize(text):
    # Handle NaN/None values or any non-string types
    if pd.isna(text) or text is None:
        return ""
    # Convert to string if it's not already a string
    if not isinstance(text, str):
        text = str(text)
    # Then proceed with normalization
    return re.sub(r'[^A-Z0-9 ]', '', text.upper())

# Function to extract Toyota model
def extract_toyota_model(text):
    txt = normalize(text)
    
    # Skip entries that clearly aren't Toyota vehicles
    non_toyota = ["TATA", "MARUTI", "HONDA", "HYUNDAI", "MAHINDRA", "TVS", "HERO"]
    for brand in non_toyota:
        if brand in txt:
            return "NOT FOUND"
    
    # First check for the most specific model combinations
    for kw in ["INNOVA CRYSTA", "COROLLA ALTIS", "ETIOS LIVA", "ETIOS CROSS", "URBAN CRUISER"]:
        if kw in txt:
            return kw
    
    # Correct aliases
    for bad, good in toyota_alias_map.items():
        if bad in txt:
            txt = txt.replace(bad, good)
    
    # Then search for any keyword - sorting by length to catch longer matches first
    for kw in sorted(toyota_model_keywords, key=len, reverse=True):
        if kw in txt:
            # For major models, try to find specific variants
            if kw == "INNOVA":
                # Look for specific Innova variants
                for variant in ["2.4G", "2.4V", "2.8G", "2.8Z", "2.5G", "2.5V", "2.5Z", "2.5E"]:
                    if variant in txt:
                        return f"INNOVA {variant}"
                return "INNOVA"
            elif kw == "FORTUNER":
                # Look for specific Fortuner variants
                for variant in ["3.0L", "2.8L", "4X4", "4WD", "LEGENDER"]:
                    if variant in txt:
                        return f"FORTUNER {variant}"
                return "FORTUNER"
            elif kw == "ETIOS":
                # Check if it's Etios Liva or regular Etios
                if "LIVA" in txt:
                    return "ETIOS LIVA"
                elif "CROSS" in txt:
                    return "ETIOS CROSS"
                # Look for specific Etios variants
                for variant in ["VD", "GD", "VX", "GX"]:
                    if variant in txt:
                        return f"ETIOS {variant}"
                return "ETIOS"
            return kw
            
    return "NOT FOUND"

# Apply extraction to the entire dataset
toyota_data['toyota_model'] = toyota_data['rc_maker_model'].apply(extract_toyota_model)

# Display extraction results
model_freq = toyota_data['toyota_model'].value_counts()
print("\nModel extraction results:")
print(model_freq.head(20))
print(f"\nModels found: {(toyota_data['toyota_model'] != 'NOT FOUND').sum()} out of {len(toyota_data)} ({(toyota_data['toyota_model'] != 'NOT FOUND').sum()/len(toyota_data)*100:.1f}%)")

# Create a final_model column using fallback logic
toyota_data['final_model'] = toyota_data['toyota_model'].apply(lambda x: x if x != "NOT FOUND" else "NOT FOUND")

# Save results to CSV for review
output_path = '/home/bipin/Documents/kotak/car_model/clean_model/toyota_models_extracted.csv'
toyota_data.to_csv(output_path, index=False)
print(f"\nSaved results to {output_path}")

# Create a frequency mapping of original values to extracted models
mapping_df = toyota_data.groupby('rc_maker_model')['toyota_model'].agg(
    lambda x: x.value_counts().index[0] if len(x.value_counts()) > 0 else "NOT FOUND"
).reset_index()
mapping_df.columns = ['original_value', 'mapped_model']
mapping_df['count'] = mapping_df['original_value'].map(toyota_data['rc_maker_model'].value_counts())
mapping_df = mapping_df.sort_values('count', ascending=False)

# Save the mapping for review
mapping_path = '/home/bipin/Documents/kotak/car_model/clean_model/toyota_model_mapping.csv'
mapping_df.to_csv(mapping_path, index=False)
print(f"Saved mapping to {mapping_path}")

# Save the keywords and alias map for reuse
with open('/home/bipin/Documents/kotak/car_model/clean_model/toyota_keywords.py', 'w') as f:
    f.write("toyota_model_keywords = [\n")
    for kw in sorted(toyota_model_keywords):
        f.write(f"    \"{kw}\",\n")
    f.write("]\n\n")
    
    f.write("toyota_alias_map = {\n")
    for bad, good in sorted(toyota_alias_map.items()):
        f.write(f"    \"{bad}\": \"{good}\",\n")
    f.write("}\n")

print("Keywords and alias map saved to toyota_keywords.py")

Loaded 7565 records from TOYOTA.csv
Top model keywords by frequency:
INNOVA: 2528
2: 2368
5: 1321
ETIOS: 962
G: 937
M: 664
V: 626
FORTUNER: 597
COROLLA: 519
1: 516
CRYSTA: 459
4: 441
LIVA: 433
8: 425
MT: 382
ALTIS: 371
VX: 360
7S: 343
GD: 342
GX: 334
8S: 306
3: 296
QUALIS: 293
E3: 264
E4: 255
AT: 249
BS4: 237
DIESEL: 213
L: 160
0L: 150

Model extraction results:
toyota_model
INNOVA           2768
FORTUNER          577
INNOVA CRYSTA     556
ETIOS LIVA        545
COROLLA ALTIS     397
ETIOS GD          360
QUALIS            318
COROLLA           312
ETIOS             303
NOT FOUND         207
ETIOS VX          136
FORTUNER 4WD      126
ETIOS VD          116
LIVA              104
YARIS              99
ETIOS CROSS        98
GLANZA             87
V                  84
CAMRY              81
URBAN CRUISER      76
Name: count, dtype: int64

Models found: 7358 out of 7565 (97.3%)

Saved results to /home/bipin/Documents/kotak/car_model/clean_model/toyota_models_extracted.csv
Saved mapping to /ho